In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib 
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("../input/loan-prediction-problem-dataset/train_u6lujuX_CVtuZ9i.csv")
df_test = pd.read_csv("../input/loan-prediction-problem-dataset/test_Y3wMUE5_7gLdaTN.csv")
df.head() 

In [ ]:
df.describe()

In [ ]:
df.info()

## Preprocessing the data and understanding what the contents of it are.

In [ ]:
df.isnull().sum()


In [ ]:
df_test.isnull().sum()

In [ ]:
#filling the missing values with the mean of the column for quantitative variables
df['LoanAmount'] = df['LoanAmount'].fillna(df['LoanAmount'].mean())
df['Loan_Amount_Term'] = df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].mean())
df['Credit_History'] = df['Credit_History'].fillna(df['Credit_History'].mean())

In [ ]:
df_test['LoanAmount'] = df_test['LoanAmount'].fillna(df_test['LoanAmount'].mean())
df_test['Loan_Amount_Term'] = df_test['Loan_Amount_Term'].fillna(df_test['Loan_Amount_Term'].mean())
df_test['Credit_History'] = df_test['Credit_History'].fillna(df_test['Credit_History'].mean())

In [ ]:
#filling the missing values with the mode of the columns for categorical variables
df['Gender'] = df['Gender'].fillna(df['Gender'].mode()[0])
df['Dependents'] = df['Dependents'].fillna(df['Dependents'].mode()[0])
df['Married'] = df['Married'].fillna(df['Married'].mode()[0])
df['Self_Employed'] = df['Self_Employed'].fillna(df['Self_Employed'].mode()[0])

In [ ]:
df_test['Gender'] = df_test['Gender'].fillna(df['Gender'].mode()[0])
df_test['Dependents'] = df_test['Dependents'].fillna(df['Dependents'].mode()[0])
df_test['Married'] = df_test['Married'].fillna(df['Married'].mode()[0])
df_test['Self_Employed'] = df_test['Self_Employed'].fillna(df['Self_Employed'].mode()[0])

In [ ]:
#checking if our code worked properly
df.isnull().sum()

## Exploratory Data Analysis

In [ ]:
sns.countplot(df['Gender'])

In [ ]:
sns.countplot(df['Dependents'])

In [ ]:
sns.countplot(df['Married'])

In [ ]:
sns.countplot(df['Self_Employed'])

In [ ]:
sns.countplot(df['Property_Area'])

In [ ]:
sns.countplot(df['Loan_Status'])

In [ ]:
##Creating this variable for future use
df['total_Income'] = df['ApplicantIncome'] + df['CoapplicantIncome']
df_test['total_Income'] = df_test['ApplicantIncome'] + df_test['CoapplicantIncome']
df['total_Income'].head()

In [ ]:
sns.distplot(df['ApplicantIncome'])

In [ ]:
#since the applicant income distribution is not normal we will convert it into normal distribution by using log transformation since normal distributions are better for training models
df['ApplicantIncome_Log'] = np.log(df['ApplicantIncome'])
sns.distplot(df['ApplicantIncome_Log'])

In [ ]:
df_test['ApplicantIncome_Log'] = np.log(df['ApplicantIncome'])

In [ ]:
#similarly for co applicant income
sns.distplot(df['CoapplicantIncome'])

In [ ]:
df['CoapplicantIncome'] = df['CoapplicantIncome']+1
df['CoapplicantIncome_Log'] = np.log(df['CoapplicantIncome'])
sns.distplot(df['CoapplicantIncome_Log'])

In [ ]:
df_test['CoapplicantIncome'] = df_test['CoapplicantIncome']+1
df_test['CoapplicantIncome_Log'] = np.log(df_test['CoapplicantIncome'])


In [ ]:
sns.displot(df['LoanAmount'])

In [ ]:
df['LoanAmount_Log'] = np.log(df['LoanAmount'])
sns.distplot(df['LoanAmount_Log'])

In [ ]:
df_test['LoanAmount_Log'] = np.log(df_test['LoanAmount'])

In [ ]:
sns.displot(df['Loan_Amount_Term'])

In [ ]:
df['Loan_Amount_Term_Log'] = np.log(df['Loan_Amount_Term'])
sns.distplot(df['Loan_Amount_Term_Log'])

In [ ]:
df_test['Loan_Amount_Term_Log'] = np.log(df_test['Loan_Amount_Term'])

In [ ]:
sns.displot(df['Credit_History'])

In [ ]:
df['total_Income_Log'] = np.log(df['total_Income'])
sns.distplot(df['total_Income_Log'])

In [ ]:
df_test['total_Income_Log'] = np.log(df_test['total_Income'])

In [ ]:
# correl
corr = df.corr()
plt.figure(figsize = (10,10))
sns.heatmap(corr, annot = True, cmap = 'GnBu' )

In [ ]:
cols = ['ApplicantIncome','CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term','Loan_ID', 'CoapplicantIncome_Log']
final = df.drop(columns = cols, axis = 1)
final_test = df_test.drop(columns = cols, axis = 1)
final

## Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
col = ['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area', 'Loan_Status']
#col = ['Dependents']
for c in col:
    final[c] = le.fit_transform(final[c])
    if (c == 'Loan_Status'):
        continue
    final_test[c] = le.fit_transform(final_test[c])

In [ ]:
final.head()

In [ ]:
final_test

### Gender: Male- 1 Female - 0
### Married: Yes - 1  No - 0
### Self_Employed : Yes - 1 No - 0
### Property_Area : Urban - 2 Rural - 0 Semi Urban - 1
### Loan_Status : Yes - 1 No - 0

# Training

In [ ]:
x_train = final.drop('Loan_Status', axis = 1)
y_train = df['Loan_Status']

# Testing part of the dataset

In [ ]:
x_test = final_test
y_test = np.empty([367,1],dtype="str")

In [ ]:
x_train = final.drop('total_Income', axis = 1)
x_train = x_train.drop('Loan_Status',axis = 1 )
x_test = x_test.drop('total_Income', axis = 1)

In [ ]:
x_test

In [ ]:
x_train['Dependents'].replace({"3+":"4"},inplace = True)
x_test['Dependents'].replace({"3+":"4"},inplace = True)

In [ ]:
x_train

In [ ]:
y_train.replace({"Y":"1","N":"0"},inplace = True)
y_train

## Model Training

In [ ]:
from sklearn.model_selection import cross_val_score
def Classify(model):
    model.fit(x_train, y_train)
    y_test = model.predict(x_test)
    return y_test

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
y_test = Classify(model)


## Final output is y_test which returns whether the loan is approved or not

In [ ]:
output = pd.DataFrame(y_test)
output

In [ ]:
output.replace({"1":"Yes","0":"No"},inplace = True)
output

In [ ]:
output.rename(columns={'':'Loan_Status'}, inplace=True)
output

In [ ]:
output.rename(columns={output.columns[0]: 'Loan_Status'}, inplace= True)
output

In [ ]:
sns.displot(output['Loan_Status'])

## this is the final prediction from the logistic regression model